In [1]:
from os import listdir, mkdir, system
from os.path import join, isfile, isdir


import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

train_folder = '/mnt/c/Users/bhaladik/mrcnn_development/training_data'

target_subfolders = ['kaggle/stage1_train', 'kromp_data']
target_subfolders_ext = ['kaggle/stage1_train', 'kaggle/stage1_test', 'kaggle/stage2_test_final', 'kromp_data', 'gunesli_data', 'korfhage_data', 'inhouse_data']
target_folders = [join(train_folder, f) for f in target_subfolders]
#target_folders_ext = target_folders = [join(train_folder, f) for f in target_subfolders_ext]
target_folders_ext = [join(train_folder, f) for f in target_subfolders_ext]
formats = ['mrcnn_format', 'kromp_format']
formats_ext = ['mrcnn_format', 'mrcnn_format','mrcnn_format', 'kromp_format', 'kromp_format', 'kromp_format', 'mrcnn_format']

In /home/bhaladik/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/bhaladik/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/bhaladik/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/bhaladik/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/bhaladik/.local/lib/python3.6/site-packages/matplotlib/

In [2]:
kaggle_exclusion_list = ['7b38c9173ebe69b4c6ba7e703c0c27f39305d9b2910f46405993d2ea7a963b80']

In [3]:
def make_stack_mask(single_mask):
    stack_mask_shape = single_mask.shape
    out_masks = []
    unique_vals = np.unique(single_mask)
    for val in unique_vals:
        if val != 0:
            mask_image = (single_mask == val).astype(np.bool)
            out_masks.append(mask_image)
    return out_masks

def adapt_masks_and_ims(bigger_image, bigger_labeled_mask, target_shape=(512, 512, 3)):
    ### since the image is reduced for the training we will not allow overlaps and create smaller images instead if we reach a border
    #print('Adapting mask and im with shapes')
    #print(bigger_image.shape)
    #print(bigger_labeled_mask.shape)
    images = []
    nested_masks = []
    for i in range(0, bigger_image.shape[0], target_shape[0]):
        for k in  range(0, bigger_image.shape[1], target_shape[1]):
            y_end = i+target_shape[0]
            x_end = k+target_shape[1]
            y_end_mask = y_end
            x_end_mask = x_end
            y_out = target_shape[0]
            x_out = target_shape[1]
            y_out_mask = target_shape[0]
            x_out_mask = target_shape[1]
            im_tile = np.zeros(target_shape, dtype=bigger_image.dtype)
            mask_tile = np.zeros((target_shape[0], target_shape[1]), dtype=bigger_labeled_mask.dtype)
            if y_end > bigger_image.shape[0]:
                rest = y_end - bigger_image.shape[0]
                #print('y Rest is {}'.format(rest))
                y_out -= rest
                y_end = bigger_image.shape[0]
            if x_end > bigger_image.shape[1]:
                rest = x_end - bigger_image.shape[1]
                #print('x Rest is {}'.format(rest))
                x_out -= rest
                x_end = bigger_image.shape[1]
            if y_end_mask > bigger_labeled_mask.shape[0]:
                rest = y_end_mask - bigger_labeled_mask.shape[0]
                #print('y Rest is {}'.format(rest))
                y_out_mask -= rest
                y_end_mask = bigger_labeled_mask.shape[0]
            if x_end_mask > bigger_labeled_mask.shape[1]:
                rest = x_end_mask - bigger_labeled_mask.shape[1]
                #print('x Rest is {}'.format(rest))
                x_out_mask -= rest
                x_end_mask = bigger_labeled_mask.shape[1]
            if len(bigger_image.shape) > 2:
                in_tile = bigger_image[i:y_end,k:x_end,:]
                im_tile[:y_out,:x_out] = in_tile
            else:
                #print(image.shape)
                single_channel_tile = bigger_image[i:y_end,k:x_end]
                #print(i)
                #print(k)
                #print(y_out)
                #print(x_out)
                #print(single_channel_tile.shape)
                im_tile[:y_out,:x_out,0] = single_channel_tile
                im_tile[:y_out,:x_out,1] = single_channel_tile
                im_tile[:y_out,:x_out,2] = single_channel_tile
            mask_tile[:y_out_mask,:x_out_mask] = bigger_labeled_mask[i:y_end_mask,k:x_end_mask]
            if (y_out > (target_shape[0] / 2.)) and (x_out > (target_shape[1] / 2.)):
                #tile = normalize_im(tile, do_sigmoid=True)
                images.append(im_tile)
                masks = make_stack_mask(mask_tile)
                nested_masks.append(masks)
    return images, nested_masks

def combine_training_data(out_folder, in_folders, formats, id_template='nuclei_{}', grayscale_threshold=10):
    use_index = 0
    if isdir(out_folder):
        system('rm -r {}'.format(out_folder))
        print('deleted previous folder')
    if not isdir(out_folder):
        mkdir(out_folder)
    object_counter = 0
    for i in range(len(in_folders)):
        in_folder = in_folders[i]
        use_format = formats[i]
        image_names = []
        if use_format == 'mrcnn_format':
            subfolders_pre = [join(in_folder, f) for f in listdir(in_folder) if isdir(join(in_folder, f))]
            subfolders = [join(in_folder, f) for f in listdir(in_folder) if isdir(join(in_folder, f)) and not (f in kaggle_exclusion_list)]
            print('Format: {} Processing {} out of {} total folders after applying exclusion list'.format(use_format, len(subfolders), len(subfolders_pre)))
            ims = []
            nested_masks = []
            for subfolder in subfolders:
                im_sub = join(subfolder, 'images')
                im_path = [join(im_sub, f) for f in listdir(im_sub) if isfile(join(im_sub, f))][0]
                
                im = cv2.imread(im_path, -1)
                #### check if image is black and white and only process black and white images
                if len(im.shape) > 2:
                    first_mean = np.mean(im[:,:,0])
                    second_mean = np.mean(im[:,:,1])
                    if (first_mean - second_mean) > grayscale_threshold:
                        """
                        print('Skipping image with name at {} becausse not grayscale'.format(im_path))
                        print('Mean values per channel are')
                        for c in range(3):
                            print(np.mean(im[:,:,c]))
                        print('total mean is {}'.format(np.mean(im)))
                        plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
                        plt.show()
                        """
                        continue
                mask_sub = join(subfolder, 'masks')
                mask_paths = [join(mask_sub, f) for f in listdir(mask_sub) if isfile(join(mask_sub, f))]
                #print('Prpcessing ' + im_path)
                #print('Creating mask images from {} files'.format(len(mask_paths)))
                masks = [cv2.imread(f, -1) for f in mask_paths]
                ims.append(im)
                image_names.append(im_path)
                nested_masks.append(masks)
        elif use_format == 'kromp_format':
            im_folder = join(in_folders[i], 'rawimages')
            seg_folder = join(in_folders[i], 'groundtruth')
            im_names = [f for f in listdir(im_folder) if isfile(join(im_folder, f))]
            ims = []
            nested_masks = []
            for k in range(len(im_names)):
                im = cv2.imread(join(im_folder, im_names[k]), -1)
                mask = cv2.imread(join(seg_folder, im_names[k]), -1)
                if (im.shape[0] > 512) and (im.shape[1] > 512): 
                    ims_here, masks_here = adapt_masks_and_ims(im, mask)
                else:
                    ims_here = [im]
                    masks_here = [make_stack_mask(mask)]
                    print('Worked on image with shape {}'.format(im.shape))
                    print('Created {} masks with first shape {} from tile of shape {}'.format(len(masks_here[0]), masks_here[0][0].shape, mask.shape))
                ims += ims_here
                image_names += [im_names[k]] * len(ims_here)
                nested_masks += masks_here
        print('Folder is {} and format is {}'.format(in_folders[i], formats[i]))
        print('Processing {} images and {} image names'.format(len(ims), len(image_names)))
        for n in range(len(ims)):
            im = ims[n]
            masks = nested_masks[n]
            object_counter += len(masks)
            if len(masks) > 0:
                out_sub_path = join(out_folder, id_template.format(use_index))
                mkdir(out_sub_path)
                #print('Writing images to new folder ' + out_sub_path)
                #print(isdir(out_sub_path))
                out_im_f = join(out_sub_path, 'images')
                out_mask_f = join(out_sub_path, 'masks')
                mkdir(out_im_f)
                mkdir(out_mask_f)
                out_im_path = join(out_im_f, id_template.format(use_index) + '.png')
                cv2.imwrite(out_im_path, im)
                for k in range(len(masks)):
                    mask = masks[k]
                    
                    if (len(mask.shape) < 2) or (mask.shape[0] < 10) or (mask.shape[1] < 10):
                        print("Found mask with strange shape of {} !!!!!".format(mask.shape))
                        print("Index is {} Format is {}".format(use_index, formats[i]))
                        print('Name is ' + image_names[n])
                        break
                    out_mask_path = join(out_mask_f, id_template.format(use_index) + '_{}.png'.format(k))
                    if mask.dtype == np.bool:
                        cv2.imwrite(out_mask_path, mask * 255)
                    else:
                        cv2.imwrite(out_mask_path, mask)
                
                if (use_index % 100) == 0:
                    print('Done with {} images'.format(use_index))
            else:
                print('STRANGEEEE')
                print('No masks found for image with shape {} at index {} in format {}'.format(im.shape, use_index, formats[i]))
            use_index += 1
        print('Done with folder ' + in_folders[i])
    print('Created dataset of {} nuclei'.format(object_counter))

In [4]:
#combine_training_data(join(train_folder, 'combined'), target_folders[::-1], formats[::-1])
combine_training_data(join(train_folder, 'combined_ext_winhous'), target_folders_ext[::-1], formats_ext[::-1])

Format: mrcnn_format Processing 11 out of 11 total folders after applying exclusion list
Folder is /mnt/c/Users/bhaladik/mrcnn_development/training_data/inhouse_data and format is mrcnn_format
Processing 11 images and 11 image names
Done with 0 images
Done with folder /mnt/c/Users/bhaladik/mrcnn_development/training_data/inhouse_data
Folder is /mnt/c/Users/bhaladik/mrcnn_development/training_data/korfhage_data and format is kromp_format
Processing 328 images and 328 image names
Done with 100 images
STRANGEEEE
No masks found for image with shape (512, 512, 3) at index 114 in format kromp_format
Done with 200 images
STRANGEEEE
No masks found for image with shape (512, 512, 3) at index 243 in format kromp_format
Done with 300 images
STRANGEEEE
No masks found for image with shape (512, 512, 3) at index 314 in format kromp_format
Done with folder /mnt/c/Users/bhaladik/mrcnn_development/training_data/korfhage_data
Folder is /mnt/c/Users/bhaladik/mrcnn_development/training_data/gunesli_data a

In [5]:
from random import shuffle

def make_val_split(in_folder, target_supfolder, train_sub='stage1_train', val_sub='stage1_test', val_split_size=0.2):
    if isdir(target_supfolder):
        system('rm -r {}'.format(target_supfolder))
        print('deleted previous folder')
    if not isdir(target_supfolder):
        mkdir(target_supfolder)
    all_subfolders = [join(in_folder, f) for f in listdir(in_folder) if isdir(join(in_folder, f))]
    shuffle(all_subfolders)
    split_index = int(len(all_subfolders) * val_split_size)
    val_paths = all_subfolders[:split_index]
    train_paths = all_subfolders[split_index:]
    print('Splitting {} total images at index {}'.format(len(all_subfolders), split_index))
    val_dir = join(target_supfolder, val_sub)
    mkdir(val_dir)
    for v_path in val_paths:
        system('cp -r {} {}'.format(v_path, val_dir))
    print('Done with validation images')
    train_dir = join(target_supfolder, train_sub)
    mkdir(train_dir)
    for t_path in train_paths:
        system('cp -r {} {}'.format(t_path, train_dir))
    return

In [7]:
make_val_split(join(train_folder, 'combined_ext_winhous'), join(train_folder, 'combined_ext_winhouse_split'))

Splitting 1476 total images at index 295
Done with validation images
